In [27]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece
!pip install transformers


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.5.1 requires packaging>=20.9, but you'll have packaging 20.4 which is incompatible.


In [28]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math
from tensorflow import keras
final_model = keras.models.load_model('baseline_model.h5')


In [29]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [79]:
import numpy as np
from tensorflow import keras

s = ['I am constantly torn between killing myself and killing everyone around me',
     'God, if ever I have come close to wanting to commit suicide, it is now',
     'Some days are just like volcanoes, your throat burns, your eyes sting, you feel like throwing up, you scream into a pillow',
     'Being with a crowd does not stop you from being depressed. in actual fact, it’s the idea that so many people exist who really don’t know what you are going through that keeps you depressed']

df = pd.DataFrame(s)
df.head()
tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
   
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])


In [80]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape


(4, 43)

In [81]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)


In [82]:
features = last_hidden_states[0][:,0,:].numpy()
print(features)


[[ 0.49015084  0.4255836  -0.23896432 ... -0.18987201  0.47397006
   0.0508057 ]
 [ 0.05615387  0.2571596  -0.11401454 ... -0.03910279  0.43000954
   0.3137861 ]
 [ 0.2749582   0.12700126 -0.12485033 ... -0.42071113  0.30415663
   0.67798656]
 [ 0.42262983  0.01419427 -0.3349677  ... -0.3781984  -0.0828277
   0.4671655 ]]


In [83]:
final_model = keras.models.load_model('baseline_model.h5')


In [84]:
predictions = final_model.predict(features)
print(predictions)


[[0.46666843]
 [0.90776426]
 [0.00533149]
 [0.00824171]]
